In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline

In [4]:
#creem una data limit, en aquest cas 1900 i la tornem un datetime per poder comparar amb la taula
datestringMin = "1990-01-01"
datestringMax = "2013-01-01"
dtMin = datetime.strptime(datestringMin, '%Y-%m-%d')
dtMax = datetime.strptime(datestringMax, '%Y-%m-%d')

#carreguem csv
#creem un DF en pandas, ficant na_values es canvia el tipus de les temperatures a float enlloc de object( degut als NaN es fica a object)
emisions = pd.read_csv("Datasets/GreenhouseEmitions/greenhouse_gas_inventory_data_data.csv", na_values = ['no info', '.'])
temperatures=pd.read_csv("Datasets/TemperaturaGlobal/GlobalLandTemperaturesByCountry.csv", na_values = ['no info', '.'])


#es descarta Chipre perque no existeix al dataset
paisos = ["Austria", "Belgium", "Bulgaria", "Croatia", "Czech Republic", "Denmark",
          "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland", "Italy", "Latvia",
          "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovakia",
          "Slovenia", "Spain", "Sweden","United Kingdom"]
tipusGas = [
  "carbon_dioxide_co2_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent",
  "nitrous_oxide_n2o_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent",
  "methane_ch4_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent"
]

tempPais={}
emisionsPais={}

In [5]:
for pais in paisos:
  arrayGasos=[]
  for tipus in tipusGas:
    emisionsGas = emisions[(emisions['category'] == tipus) & (emisions["country_or_area"] == pais) & (emisions['year'] < 2013)]
    emisionsGas = emisionsGas[['year','value']]
    emisionsGas = emisionsGas.sort_values(by = ['year'])
    punts = []
    #iterar sobre cada fila del dataframe guardant dins d'una llista de tuples que seran punts per interpolar, arrodonint els kt de Co2 a 3 decimals
    for index,row in emisionsGas.iterrows():
      punts.append((row['year'], round(row['value'],3)))
    xi, yi = zip(*punts)
    #numero de punts que em de treure amb la interpolacio
    x = np.linspace(min(xi), max(xi), num=len(punts)*12)
    ysp = InterpolatedUnivariateSpline(xi, yi)(x)

    #per cada valor del array resultant d'interpolar ho relacionem amb un mes any.mes
    anyInicial = 1990
    mesInicial = 0.00
    gasxMes = []
    for mostra in ysp:
      if (mesInicial > 0.11):
        anyInicial+= 1
        mesInicial = 0.01
      else:
        mesInicial+= 0.01
      if (tipus == "carbon_dioxide_co2_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent"):
        gasxMes.append((anyInicial+mesInicial,mostra,pais,1))
      elif(tipus == "nitrous_oxide_n2o_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent"):
        gasxMes.append((anyInicial+mesInicial,mostra,pais,2))
      elif(tipus == "methane_ch4_emissions_without_land_use_land_use_change_and_forestry_lulucf_in_kilotonne_co2_equivalent"):
        gasxMes.append((anyInicial+mesInicial,mostra,pais,3))
    #creem el dataset indicant el nom de les columnes i amb l'array de tuples que s'ha fet en el for
    labels = ['dt','Value','Country','Type']
    emisionsGas=pd.DataFrame.from_records(gasxMes, columns=labels)
    arrayGasos.append(emisionsGas)
  emisionsTotals=pd.concat(arrayGasos, ignore_index=True)
  emisionsPais[pais]=emisionsTotals
  arrayGasos=[]

In [6]:
for pais in paisos:
  #fa una copia del DF original amb les columnes que s'indiquen
  temperaturesPais = temperatures[['Country','AverageTemperature','dt']]
  #convertim les dates a datetime per poder agafar les que ens interessin
  temperaturesPais[['dt']] = temperaturesPais[['dt']].apply(pd.to_datetime)
  #ens centrem en un pais en concret en anys més elevats que 1900
  temperaturesPais = temperaturesPais[(temperaturesPais['dt'] >= dtMin) & (temperaturesPais['Country'] == pais) & (temperaturesPais['dt'] < dtMax)]
  #Fent aixo no sabem si esta copiant o substituint
  #dfGer['dt'].dt.strftime('%Y')
  #amb el loc si que sabem que substitueix a part de fer menys accessos a les dades
  temperaturesPais.loc[:,'dt'] = temperaturesPais.loc[:,'dt'].dt.strftime('%Y')
  
  #convertim les dates de string a float
  temperaturesPais['dt'] = temperaturesPais.dt.astype('float')
  #fiquem el mes com si fossin decimals
  anyInicial = 1990
  mesInicial = 0.00
  for index, row in temperaturesPais.iterrows():
    if (mesInicial > 0.11):
      mesInicial = 0.01
    else:
      mesInicial+= 0.01
    temperaturesPais.loc[index,'dt'] = float(row['dt'])
    temperaturesPais.loc[index,'dt'] = row['dt']+mesInicial
  tempPais[pais] = temperaturesPais
  print (pais)

E:\Anaconda\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


Austria
Belgium
Bulgaria
Croatia
Czech Republic
Denmark
Estonia
Finland
France
Germany
Greece
Hungary
Ireland
Italy
Latvia
Lithuania
Luxembourg
Malta
Netherlands
Poland
Portugal
Romania
Slovakia
Slovenia
Spain
Sweden
United Kingdom


In [10]:
#fem merge dels datasets de temperatures i el de emisions
result = pd.merge(emisionsPais["Germany"], tempPais["Germany"], on=['dt','Country'])
dadesEuropa=[]
for pais in emisionsPais:
  dadesEuropa.append(pd.merge(emisionsPais[pais], tempPais[pais], on=['dt','Country']))
result=pd.concat(dadesEuropa, ignore_index=True)

mes = 1
cops = 1

for index in range(0,len(result)):
  if mes < 13:
    if cops < 4:
      result.at[index,'Mes'] = mes
      cops += 1
    else:
      cops = 1
      mes+=1
      if cops < 4:
        result.at[index,'Mes'] = mes
        cops += 1
  else:
    mes = 1
    if cops < 4:
      result.at[index,'Mes'] = mes
      cops += 1

result['dt'] = result.dt.astype('int')
result['Mes'] = result.Mes.astype('int')
#Canviem els paisos per numeros
Npais = 1
for pais in paisos:
    result.Country[result.Country == pais] = Npais
    Npais += 1

result.to_csv('DataSetFinal.csv')
print (result)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


         dt          Value Country  Type  AverageTemperature  Mes
0      1990   62297.166000       1     1              -2.443    1
1      1990    4293.063000       1     2              -2.443    1
2      1990   10598.566000       1     3              -2.443    1
3      1990   63417.203939       1     1               2.713    2
4      1990    4342.948570       1     2               2.713    2
5      1990   10608.935464       1     3               2.713    2
6      1990   64354.776643       1     1               5.391    3
7      1990    4385.020950       1     2               5.391    3
8      1990   10614.433618       1     3               5.391    3
9      1990   65120.698769       1     1               5.462    4
10     1990    4419.699865       1     2               5.462    4
11     1990   10615.394522       1     3               5.462    4
12     1990   65725.784968       1     1              12.561    5
13     1990    4447.405043       1     2              12.561    5
14     199